In [1]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)


True

In [3]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [4]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

In 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent, autonomous systems. LangChain stands out with its modular design, enabling developers to integrate various components seamlessly. It supports the creation of complex workflows by chaining together different models and data sources, facilitating sophisticated and context-aware interactions. LangChain's extensive support for third-party integrations, from cloud providers to search engines, makes it highly adaptable to a wide range of applications such as conversational agents, document analysis, and code generation. ([dev.to](https://dev.to/surgedatalab/best-5-frameworks-for-agentic-ai-in-2025-enabling-next-gen-intelligent-multi-agent-systems-40ce?utm_source=openai))

Another notable framework is LangGraph, an extension of LangChain that introduces a graph-based system for managing the flow of data between agents. This approach is particularly useful for applications where decisions can loop or change paths depending on new input. LangGraph offers agent orchestration using graph-based logic, support for branching, looping, and conditional flows, and built-in support for asynchronous task execution. It is ideal for designing systems that mimic human decision-making in areas like loan processing, insurance claims, or workflow automation. ([phyniks.com](https://phyniks.com/blog/top-7-agentic-ai-frameworks-in-2025?utm_source=openai))

AutoGen, developed by Microsoft, specializes in orchestrating multiple AI agents to form autonomous, event-driven systems capable of handling complex, multi-agent tasks seamlessly. It provides a conversation-based coordination framework, built-in agents, and easy prototyping tools. AutoGen is compatible with OpenAI and Azure OpenAI endpoints, making it suitable for enterprise-level applications needing event-driven workflows, such as customer service automation. ([linkedin.com](https://www.linkedin.com/pulse/ai-agent-frameworks-june-2025-comprehensive-overview-chadi-abi-fadel-wcu5c?utm_source=openai))

CrewAI adopts a role-based agent collaboration approach, facilitating the creation of specialized agents that collaborate on complex projects akin to a team environment. It offers dynamic task planning, real-time performance monitoring, and orchestration of a variety of agents as distinct workers. CrewAI is ideal for complex projects requiring teamwork among agents, suitable for scenarios like software development or project management. ([linkedin.com](https://www.linkedin.com/pulse/ai-agent-frameworks-june-2025-comprehensive-overview-chadi-abi-fadel-wcu5c?utm_source=openai))

Additionally, Amazon Web Services (AWS) introduced Amazon Bedrock AgentCore, a platform designed to simplify the development and deployment of advanced AI agents. AgentCore includes modular services such as AgentCore Runtime for scalable serverless deployment, AgentCore Memory for context management, and AgentCore Identity for secure service access. This suite aims to streamline the adoption and production deployment of AI agents, marking a significant shift in how software interacts with the real world. ([techradar.com](https://www.techradar.com/pro/aws-looks-to-super-charge-ai-agents-with-amazon-bedrock-agentcore?utm_source=openai))

These frameworks represent the forefront of AI agent development in 2025, each contributing to the evolution of intelligent, autonomous systems across various industries. 

In [9]:
# Websearch tool has gigher cost 25$ for 1000 searches.
HOW_MANY_SEARCHES = 1

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# With massive thanks to student Wes C. for discovering and fixing a nasty bug with this!

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [10]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)




searches=[WebSearchItem(reason='To gather the most up-to-date information on AI agent frameworks that are emerging or being developed in 2025.', query='latest AI agent frameworks 2025')]


In [11]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("abhishek@textraja.com") # Change this to your verified email
    to_email = To("samikshabagda1996@gmail.com") # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [12]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given subject and HTML body', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x120e17ec0>, strict_json_schema=True, is_enabled=True)

In [13]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)


In [16]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [17]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

In [19]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("done!")


Starting research...
Planning searches...
Will perform 1 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Writing email...
Email sent
done!


In [20]:
report

ReportData(short_summary="As of 2025, various innovative AI agent frameworks have emerged, significantly enhancing the deployment and functionality of autonomous systems. Key frameworks include Amazon Bedrock AgentCore, OpenAI's ChatGPT Agent, and the Model Context Protocol (MCP), each contributing to improved accessibility, scalability, and interoperability across the AI ecosystem.", markdown_report="# Latest AI Agent Frameworks in 2025\n\n## Table of Contents\n1. Introduction  \n2. Overview of AI Agent Frameworks  \n    - 2.1 Amazon Bedrock AgentCore  \n    - 2.2 OpenAI's ChatGPT Agent  \n    - 2.3 Model Context Protocol (MCP)  \n3. Academic Innovations  \n    - 3.1 AutoAgent  \n    - 3.2 Eliza  \n4. Implications of AI Agent Frameworks  \n5. Conclusion  \n6. References  \n\n---  \n\n## 1. Introduction\nThe paradigm of artificial intelligence (AI) is witnessing rapid advancements, particularly in the development of AI agents that can autonomously execute tasks. By 2025, several framew